In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import math
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import numpy as np
from pymle.sim.Simulator1D import Simulator1D
from pymle.core.TransitionDensity import ExactDensity, KesslerDensity
from pymle.fit.AnalyticalMLE import AnalyticalMLE
from pymle.models import GeometricBM, OrnsteinUhlenbeck
from sklearn.linear_model import LinearRegression
from scipy.optimize import brute
from decimal import Decimal, ROUND_HALF_UP
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from statsmodels.tsa.statespace.structural import UnobservedComponents
from scipy.interpolate import UnivariateSpline
from statsmodels.graphics.tsaplots import plot_acf
from scipy.signal import periodogram, find_peaks
import itertools
from scipy.optimize import minimize, Bounds, LinearConstraint
from joblib import Parallel, delayed
import os
import time

Round 3

In [ ]:
containers = [
    # Row A (Left to Right)
    (80, 6), (50, 4), (83, 7), (31, 2), (60, 4),
    # Row B (Left to Right)
    (89, 8), (10, 1), (37, 3), (70, 4), (90, 10),
    # Row C (Left to Right)
    (17, 1), (40, 3), (73, 4), (100, 15), (20, 2),
    # Row D (Left to Right)
    (41, 3), (79, 5), (23, 2), (47, 3), (30, 2)
]

arr_containers = np.array(containers)

B = 10000
F2 = 50000 
F3 = 100000
F_total_3 = F2 + F3

In [ ]:
def maximin_1(container_array, B):
    max_metric_val = -float('inf')
    argmax = []
    for i in range(len(container_array)): 
        mult, inhab = container_array[i] 
        metric_val = mult / (inhab + 100) 
        if math.isclose(metric_val, max_metric_val):
            argmax.append((mult, inhab))
        elif metric_val > max_metric_val:
            argmax = [(mult, inhab)]
            max_metric_val = metric_val
    max_profit_val = max_metric_val * B
    return argmax, max_profit_val

In [ ]:
def _objective_f_2d(p_vec, m1, i1, m2, i2):
    p1, p2 = p_vec
    # Add epsilon for numerical stability
    eps = 1e-9
    # Assume i1, i2 are >= 1 based on container data
    term1 = m1 / (i1 + 100 * p1 + eps)
    term2 = m2 / (i2 + 100 * p2 + eps)
    return term1 + term2

def maximin_2(container_array, B, F):
    max_min_profit = -float('inf') 
    best_pairs = [] 

    # Bounds: p1 >= 0, p2 >= 0 (upper bound 1 is implicitly handled by p1+p2<=1)
    bounds = Bounds([0.0, 0.0], [1.0, 1.0]) 
    # Linear Constraint: p1 + p2 <= 1 --> 1*p1 + 1*p2 <= 1
    linear_constraint = LinearConstraint([[1, 1]], [-np.inf], [1.0]) 

    for (m1, i1), (m2, i2) in itertools.combinations(container_array, 2):
        # Initial guess for the optimizer (e.g., start near feasible center)
        initial_guess = [0.5, 0.5] 
        # Ensure guess respects sum constraint if starting there is important
        if sum(initial_guess) > 1: initial_guess = [0.5/sum(initial_guess), 0.5/sum(initial_guess)]

        # Perform 2D constrained minimization
        result = minimize(
            _objective_f_2d, 
            initial_guess, 
            args=(m1, i1, m2, i2), # Pass fixed M, I parameters
            method='SLSQP',
            bounds=bounds, 
            constraints=[linear_constraint]
        )
        
        min_summed_ratio = result.fun 
            
        # Calculate the maximin profit for this pair
        current_min_profit = (min_summed_ratio * B) - F 
        
        # Update tracker for the overall maximin profit
        current_pair_tuple = tuple(sorted([(m1, i1), (m2, i2)]))
        if math.isclose(current_min_profit, max_min_profit):
             if current_pair_tuple not in {tuple(p) for p in best_pairs}:
                 best_pairs.append(list(current_pair_tuple))
        elif current_min_profit > max_min_profit:
            best_pairs = [list(current_pair_tuple)] 
            max_min_profit = current_min_profit
            
    final_best_pairs = [tuple(pair) for pair in best_pairs]
    if max_min_profit == -float('inf'): return [], -float('inf') 
    return final_best_pairs, max_min_profit

In [ ]:
def _objective_f_3d(p_vec, m1, i1, m2, i2, m3, i3):
    p1, p2, p3 = p_vec
    eps = 1e-9
    term1 = m1 / (i1 + 100 * p1 + eps)
    term2 = m2 / (i2 + 100 * p2 + eps)
    term3 = m3 / (i3 + 100 * p3 + eps)
    return term1 + term2 + term3

def maximin_3(container_array, B, F_cost_total_3): # Use F_cost_total_3 (150k)
    max_min_profit = -float('inf')
    best_triplets = []

    # Bounds: p1, p2, p3 >= 0 (upper bound 1 implicitly handled by p1+p2+p3<=1)
    bounds = Bounds([0.0, 0.0, 0.0], [1.0, 1.0, 1.0])
    # Linear Constraint: p1 + p2 + p3 <= 1 --> 1*p1 + 1*p2 + 1*p3 <= 1
    linear_constraint = LinearConstraint([[1, 1, 1]], [-np.inf], [1.0])

    # Iterate through all combinations of 3 containers
    for (m1, i1), (m2, i2), (m3, i3) in itertools.combinations(container_array, 3):
        # Initial guess for the optimizer (e.g., start near feasible center)
        initial_guess = [1/3, 1/3, 1/3]
         # Ensure guess respects sum constraint if starting there is important
        if sum(initial_guess) > 1:
            initial_guess = [g / sum(initial_guess) for g in initial_guess]


        result = minimize(
            _objective_f_3d,
            initial_guess,
            args=(m1, i1, m2, i2, m3, i3), 
            method='SLSQP',
            bounds=bounds,
            constraints=[linear_constraint]
        )

        if result.success:
            min_summed_ratio = result.fun

            current_min_profit = (min_summed_ratio * B) - F_cost_total_3

            current_triplet_tuple = tuple(sorted([(m1, i1), (m2, i2), (m3, i3)]))

            if math.isclose(current_min_profit, max_min_profit, rel_tol=1e-9, abs_tol=1e-9):
                 if current_triplet_tuple not in {tuple(p) for p in best_triplets}:
                      best_triplets.append(list(current_triplet_tuple)) 
            elif current_min_profit > max_min_profit:
                best_triplets = [list(current_triplet_tuple)]
                max_min_profit = current_min_profit

    final_best_triplets = [tuple(triplet) for triplet in best_triplets]
    if max_min_profit == -float('inf'): return [], -float('inf')

    return final_best_triplets, max_min_profit

In [ ]:
def breakeven_rate(container_tuple):
  multiplier, inhabitants = container_tuple
  pk_breakeven_2nd = max(0.0, (multiplier / 5.0 - inhabitants) / 100.0)
  pk_breakeven_3rd = max(0.0, (multiplier / 10.0 - inhabitants) / 100.0)
  return (pk_breakeven_2nd, pk_breakeven_3rd)

all_breakeven_rates = [breakeven_rate(tuple(container)) for container in arr_containers]
print(all_breakeven_rates)
for i, rates_tuple in enumerate(all_breakeven_rates):
    original_container = tuple(arr_containers[i]) 
    print(f"Container {original_container}:")
    print(f"  Breakeven % as 2nd: {rates_tuple[0] * 100:.2f}%")
    print(f"  Breakeven % as 3rd: {rates_tuple[1] * 100:.2f}%")

arr_breakeven_rates = np.array(all_breakeven_rates)
arr_breakeven_rates

[(0.1, 0.02), (0.06, 0.01), (0.09600000000000002, 0.013000000000000006), (0.042, 0.011000000000000001), (0.08, 0.02), (0.098, 0.009000000000000003), (0.01, 0.0), (0.044000000000000004, 0.007000000000000002), (0.1, 0.03), (0.08, 0.0), (0.024, 0.006999999999999999), (0.05, 0.01), (0.106, 0.033), (0.05, 0.0), (0.02, 0.0), (0.05199999999999999, 0.010999999999999996), (0.10800000000000001, 0.029000000000000005), (0.025999999999999995, 0.0029999999999999983), (0.064, 0.017), (0.04, 0.01)]
Container (80, 6):
  Breakeven % as 2nd: 10.00%
  Breakeven % as 3rd: 2.00%
Container (50, 4):
  Breakeven % as 2nd: 6.00%
  Breakeven % as 3rd: 1.00%
Container (83, 7):
  Breakeven % as 2nd: 9.60%
  Breakeven % as 3rd: 1.30%
Container (31, 2):
  Breakeven % as 2nd: 4.20%
  Breakeven % as 3rd: 1.10%
Container (60, 4):
  Breakeven % as 2nd: 8.00%
  Breakeven % as 3rd: 2.00%
Container (89, 8):
  Breakeven % as 2nd: 9.80%
  Breakeven % as 3rd: 0.90%
Container (10, 1):
  Breakeven % as 2nd: 1.00%
  Breakeven % 

array([[0.1  , 0.02 ],
       [0.06 , 0.01 ],
       [0.096, 0.013],
       [0.042, 0.011],
       [0.08 , 0.02 ],
       [0.098, 0.009],
       [0.01 , 0.   ],
       [0.044, 0.007],
       [0.1  , 0.03 ],
       [0.08 , 0.   ],
       [0.024, 0.007],
       [0.05 , 0.01 ],
       [0.106, 0.033],
       [0.05 , 0.   ],
       [0.02 , 0.   ],
       [0.052, 0.011],
       [0.108, 0.029],
       [0.026, 0.003],
       [0.064, 0.017],
       [0.04 , 0.01 ]])

In [ ]:
maximin1_choice, maximin1_profit = maximin_1(arr_containers, B)
maximin2_choice, maximin2_profit = maximin_2(arr_containers, B, F2)
maximin3_choice, maximin3_profit = maximin_3(arr_containers, B, F_total_3)

# --- Output ---
print(f"Maximin choice (1 container): {maximin1_choice}")
print(f"Maximin profit (worst-case, 1 container): {maximin1_profit:.2f}")
print("-" * 30)
print(f"Maximin choice(s) (2 containers): {maximin2_choice}")
print(f"Maximin profit (worst-case, 2 containers): {maximin2_profit:.2f}")
print("-" * 30)
print(f"Maximin choice(s) (3 containers): {maximin3_choice}")
print(f"Maximin profit (worst-case, 3 containers): {maximin3_profit:.2f}")
print("-" * 30)

Maximin choice (1 container): [(100, 15)]
Maximin profit (worst-case, 1 container): 8695.65
------------------------------
Maximin choice(s) (2 containers): [((89, 8), (100, 15))]
Maximin profit (worst-case, 2 containers): -19294.34
------------------------------
Maximin choice(s) (3 containers): [((89, 8), (90, 10), (100, 15))]
Maximin profit (worst-case, 3 containers): -87111.77
------------------------------
